# tf.train.Example and TFRecord

TFRecord is a sequence of tf.train.Example as a binary record.

In [1]:
from typing import (
    List
)
import json
import tensorflow as tf

In [2]:
!rm -rf ./data/
!mkdir -p ./data/protobuf
!mkdir -p ./data/tfrecord

# Protobuf
## References

* [Protocol Buffer documentation](https://developers.google.com/protocol-buffers/docs/overview)
* [Protocol Buffer Basics: Python](https://developers.google.com/protocol-buffers/docs/pythontutorial)

> * Define message formats in a .proto file.
> * Use the protocol buffer compiler.
> * Use the Python protocol buffer API to write and read messages.

* [Protocol Buffers Python API Reference (Python)](https://googleapis.dev/python/protobuf/latest/index.html)
* [Protobuf parsing in Python](https://www.datadoghq.com/blog/engineering/protobuf-parsing-in-python/)

## Define proto file

In [3]:
%%writefile metric.proto
syntax = "proto3";
message Metric {
  string name = 1;
  string type = 2;
  float value = 3;
}

Overwriting metric.proto


In [4]:
!cat metric.proto

syntax = "proto3";
message Metric {
  string name = 1;
  string type = 2;
  float value = 3;
}


## Generate python client

In [5]:
!rm metric_pb2.py

In [6]:
!protoc --python_out=. metric.proto

In [7]:
!ls

__pycache__  metric.proto   tf_create_tf_record.ipynb  tfx
data	     metric_pb2.py  tf_example.ipynb


## Write protobuf file

In [8]:
import metric_pb2

my_metric = metric_pb2.Metric()
my_metric.name = 'sys.cpu'
my_metric.type = 'gauge'
my_metric.value = 99.9

with open('./data/protobuf/metric.bin', 'wb') as f:
    f.write(my_metric.SerializeToString())

## Read protobuf file

In [9]:
with open('./data/protobuf/metric.bin', 'rb') as f:
    metric = metric_pb2.Metric()
    metric.ParseFromString(f.read())

print(type(metric))
print(dir(metric))

<class 'metric_pb2.Metric'>
['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'RegisterExtension', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', '_CheckCalledFromGeneratedFile', '_SetListener', '__class__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__unicode__', '_extensions_by_name', '_extensions_by_number', 'name', 'type', 'value']


In [10]:
for field in metric.DESCRIPTOR.fields:
    print(field.full_name)

Metric.name
Metric.type
Metric.value


In [11]:
[field.name for field in metric.DESCRIPTOR.fields]

['name', 'type', 'value']

In [12]:
metric.ListFields()

[(<google.protobuf.pyext._message.FieldDescriptor at 0x7fe3160ea410>,
  'sys.cpu'),
 (<google.protobuf.pyext._message.FieldDescriptor at 0x7fe3160ea290>, 'gauge'),
 (<google.protobuf.pyext._message.FieldDescriptor at 0x7fe3160eef50>,
  99.9000015258789)]

# tf.train.Example

* [TFRecord and tf.train.Example ](https://www.tensorflow.org/tutorials/load_data/tfrecord)

> The tf.train.Example message (**or protobuf**) is a flexible message type that represents a {"string": value} mapping. It is designed for use with TensorFlow and is used throughout the higher-level APIs such as TFX.

> Fundamentally, a tf.train.Example is a {"string": tf.train.Feature} mapping. The tf.train.Feature message type can accept one of the following three types (See the .proto file for reference). Most other generic types can be coerced into one of these:  
> 1. tf.train.BytesList (the following types can be coerced)
> * string
> * byte
> 2. tf.train.FloatList (the following types can be coerced)
> * float (float32)
> * double (float64)
> 3. tf.train.Int64List (the following types can be coerced)
> * bool
> * enum
> * int32
> * uint32
> * int64
> * uint64

## Utility

* [tf.io.serialize_tensor ](https://www.tensorflow.org/api_docs/python/tf/io/serialize_tensor)

> useful in converting multidimensional Tensor into a format accepted by binary storage formats such as a TFRecord or tf.train.Example.

* [tf.io.decode_json_example](https://www.tensorflow.org/api_docs/python/tf/io/decode_json_example)

> Convert JSON-serialized tf.train.Example (e.g created with ```google.protobuf.json_format.MessageToJson```) to a binary-serialized tf.train.Example (equivalent to Example.SerializeToString()) suitable for conversion to tensors with tf.io.parse_example.



In [13]:
# tf.train.Example is a protobuf message
example_as_protobuf_message = tf.train.Example(
    features=tf.train.Features(feature={
        "a": tf.train.Feature(int64_list=tf.train.Int64List(value=[1, 1, 3]))
    })
)

from google.protobuf import json_format
example_json = json_format.MessageToJson(example_as_protobuf_message)
print(example_json)

{
  "features": {
    "feature": {
      "a": {
        "int64List": {
          "value": [
            "1",
            "1",
            "3"
          ]
        }
      }
    }
  }
}


In [14]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Metric to tf.train.Example

In [15]:
def metric_to_example(metric_name:str, metric_type:str, metric_value: float):
    """
    Creates a tf.train.Example message ready to be written to a file for the metric data
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
        'name': _bytes_feature(metric_name),
        'type': _bytes_feature(metric_type),
        'value': _float_feature(metric_value),
    }

    # Create a Features message using tf.train.Example.
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example

def serialize_example(example:tf.train.Example):
    return example.SerializeToString(example)


def deserialize_example_to_metric(serialized_example: str):
    return tf.train.Example.FromString(serialized_example)

In [16]:
example = metric_to_example(
    metric_name='sys.cpu',
    metric_type='gauge',
    metric_value=99.9
)
example

features {
  feature {
    key: "name"
    value {
      bytes_list {
        value: "sys.cpu"
      }
    }
  }
  feature {
    key: "type"
    value {
      bytes_list {
        value: "gauge"
      }
    }
  }
  feature {
    key: "value"
    value {
      float_list {
        value: 99.9000015258789
      }
    }
  }
}

In [17]:
import google.protobuf as protobuf

parsed = json.loads(
    protobuf.json_format.MessageToJson(example)
)
print(json.dumps(parsed, indent=2, sort_keys=True))

{
  "features": {
    "feature": {
      "name": {
        "bytesList": {
          "value": [
            "c3lzLmNwdQ=="
          ]
        }
      },
      "type": {
        "bytesList": {
          "value": [
            "Z2F1Z2U="
          ]
        }
      },
      "value": {
        "floatList": {
          "value": [
            99.9
          ]
        }
      }
    }
  }
}


## Write Examples to TFRecord file

---
# TFX

* [The ExampleGen TFX Pipeline Component ](https://www.tensorflow.org/tfx/guide/examplegen)

In [18]:
def write_examples_to_tf_record_file(
    path_to_tfrecord_file:str, 
    examples: List[tf.train.Example]
):
    # Write the `tf.train.Example` observations to the file.
    with tf.io.TFRecordWriter(path_to_tfrecord_file) as w:
        for e in examples:
            w.write(serialize_example(example))

In [19]:
write_examples_to_tf_record_file('./data/tfrecord/records.tfr', [example])

## Read from TFReord file

In [21]:
!rm -rf ./data/tfx
!mkdir ./data/tfx

In [22]:
import os
from tfx import v1 as tfx
from tfx.components import ImportExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

context = InteractiveContext(pipeline_root='./data/tfx')

In [23]:
example_gen = ImportExampleGen(input_base="./data/tfrecord")
result = context.run(example_gen)

In [24]:
result

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [25]:
component = result.component
component

ImportExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7fe2ffa6e450>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7fe3160d1850>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: ImportExampleGen, inputs: {}, outputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={})})

In [26]:
dir(component)

['DRIVER_CLASS',
 'EXECUTOR_SPEC',
 'SPEC_CLASS',
 '_CONSTRUCT_ARGS',
 '_CONSTRUCT_CLS',
 '_CONSTRUCT_KWARGS',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_pip_dependency',
 '_downstream_nodes',
 '_id',
 '_pip_dependencies',
 '_resolve_pip_dependencies',
 '_upstream_nodes',
 '_validate_component_class',
 '_validate_spec',
 'add_downstream_node',
 'add_downstream_nodes',
 'add_upstream_node',
 'add_upstream_nodes',
 'component_id',
 'component_type',
 'downstream_nodes',
 'driver_class',
 'exec_properties',
 'executor_spec',
 'from_json_dict',
 'get_class_type',
 'id',
 'inputs',
 'outputs',
 'platform_config'

In [27]:
for artifact in component.outputs['examples'].get():
    print(artifact)

Artifact(artifact: id: 1
type_id: 14
uri: "./data/tfx/ImportExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:77,xor_checksum:1654937053,sum_checksum:1654937053"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.8.0"
  }
}
state: LIVE
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
base_type: DATASET
)
